In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from scipy.stats import f
%matplotlib inline

In [2]:
sm.__version__

'0.12.1'

In [3]:
df = pd.read_csv('https://web.stanford.edu/~hastie/ElemStatLearn/datasets/prostate.data',sep='\t')
df.head()

Unnamed: 0    lcavol   lweight  age      lbph  svi       lcp  gleason  \
0           1 -0.579818  2.769459   50 -1.386294    0 -1.386294        6   
1           2 -0.994252  3.319626   58 -1.386294    0 -1.386294        6   
2           3 -0.510826  2.691243   74 -1.386294    0 -1.386294        7   
3           4 -1.203973  3.282789   58 -1.386294    0 -1.386294        6   
4           5  0.751416  3.432373   62 -1.386294    0 -1.386294        6   

   pgg45      lpsa train  
0      0 -0.430783     T  
1      0 -0.162519     T  
2     20 -0.162519     T  
3      0 -0.162519     T  
4      0  0.371564     T

In [72]:
df_to_work= df[df.train=='T'].copy()
cols_to_select = ['lcavol','lweight','age','lbph','svi','lcp','gleason','pgg45']
X_train = df_to_work.loc[:,cols_to_select]
Y_train = df[df.train=='T'].loc[:,['lpsa']].copy()
Y_train.reset_index(drop=True,inplace=True)
display(X_train.corr())
scaler = StandardScaler()
X_transformed = pd.DataFrame(scaler.fit_transform(X_train),columns=cols_to_select)
X_transformed = sm.add_constant(X_transformed)
model = sm.OLS(Y_train,X_transformed)
results = model.fit()
rss1 = results.ssr
updated_cols_to_select = list(results.pvalues[results.pvalues < 0.05].index)
updated_cols_to_select.remove('const')
print(updated_cols_to_select)

lcavol   lweight       age      lbph       svi       lcp   gleason  \
lcavol   1.000000  0.300232  0.286324  0.063168  0.592949  0.692043  0.426414   
lweight  0.300232  1.000000  0.316723  0.437042  0.181054  0.156829  0.023558   
age      0.286324  0.316723  1.000000  0.287346  0.128902  0.172951  0.365915   
lbph     0.063168  0.437042  0.287346  1.000000 -0.139147 -0.088535  0.032992   
svi      0.592949  0.181054  0.128902 -0.139147  1.000000  0.671240  0.306875   
lcp      0.692043  0.156829  0.172951 -0.088535  0.671240  1.000000  0.476437   
gleason  0.426414  0.023558  0.365915  0.032992  0.306875  0.476437  1.000000   
pgg45    0.483161  0.074166  0.275806 -0.030404  0.481358  0.662533  0.757056   

            pgg45  
lcavol   0.483161  
lweight  0.074166  
age      0.275806  
lbph    -0.030404  
svi      0.481358  
lcp      0.662533  
gleason  0.757056  
pgg45    1.000000

['lcavol', 'lweight', 'lbph', 'svi']


In [86]:
df_to_work= df[df.train=='T'].copy()
X_train = df_to_work.loc[:,updated_cols_to_select]
Y_train = df[df.train=='T'].loc[:,['lpsa']].copy()
Y_train.reset_index(drop=True,inplace=True)
display(X_train.corr())
scaler = StandardScaler()
X_transformed = pd.DataFrame(scaler.fit_transform(X_train),columns=updated_cols_to_select)
X_transformed = sm.add_constant(X_transformed)
model = sm.OLS(Y_train,X_transformed)
results = model.fit()
rss0 = results.ssr
rss0

lcavol   lweight      lbph       svi
lcavol   1.000000  0.300232  0.063168  0.592949
lweight  0.300232  1.000000  0.437042  0.181054
lbph     0.063168  0.437042  1.000000 -0.139147
svi      0.592949  0.181054 -0.139147  1.000000

32.81499474881555

In [87]:
results.rsquared

0.6591763372202288

In [85]:
results.aic

152.31269099964987

In [82]:
p1 = len(cols_to_select)
p0 = len(updated_cols_to_select)
N=df_to_work.shape[0]
F_stat = (N- p1 -1)*(rss0 - rss1)/((p1 - p0)*rss1)
print(1- f.cdf(F_stat,p1-p0,N - p1 -1 ))

0.16933707265225173
